In [56]:
%pip install "zenml"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

Note: you may need to restart the kernel to use updated packages.
⠏ Installing integrations.....
Note: you may need to restart the kernel to use updated packages.


{'status': 'ok', 'restart': True}

In [57]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("zenml_password")
secret_value_1 = user_secrets.get_secret("zenml_user")
secret_value_2 = user_secrets.get_secret("wandb")

In [58]:
import os
os.system("zenml connect --url 'https://bsenst-zenml-server.hf.space/' --username="+secret_value_1+" --password="+secret_value_0)

Connecting to: 'https://bsenst-zenml-server.hf.space/'...
Updated the global store configuration.


0

In [2]:
! zenml status

Using configuration from: '/root/.config/zenml'
Local store files are located at: '/root/.config/zenml/local_stores'
Active repository root: /kaggle/working
Connected to a ZenML server: 'https://bsenst-zenml-server.hf.space'
The current user is: 'Benjamin'
The active workspace is: 'default' (repository)
The active stack is: 'mlflow_stack' (repository)


In [3]:
# https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
! wget --directory-prefix=input https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
! wget --directory-prefix=input https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
! wget --directory-prefix=input https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet

--2023-05-24 19:05:39--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.160.213, 52.84.160.116, 52.84.160.84, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.160.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1254291 (1.2M) [binary/octet-stream]
Saving to: ‘input/green_tripdata_2022-01.parquet.1’

green_tripdata_2022 100%[===================>]   1.20M  --.-KB/s    in 0.03s   

2023-05-24 19:05:40 (36.9 MB/s) - ‘input/green_tripdata_2022-01.parquet.1’ saved [1254291/1254291]

--2023-05-24 19:05:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.160.116, 52.84.160.213, 52.84.160.84, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.160.116|:443... connected.
HT

In [4]:
! wget https://github.com/DataTalksClub/mlops-zoomcamp/raw/main/cohorts/2023/02-experiment-tracking/homework/preprocess_data.py

--2023-05-24 19:05:43--  https://github.com/DataTalksClub/mlops-zoomcamp/raw/main/cohorts/2023/02-experiment-tracking/homework/preprocess_data.py
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2023/02-experiment-tracking/homework/preprocess_data.py [following]
--2023-05-24 19:05:43--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2023/02-experiment-tracking/homework/preprocess_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2512 (2.5K) [text/plain]
Saving to: ‘preprocess_data.py.1’

preprocess_data.py. 100%[====

In [5]:
! python preprocess_data.py --raw_data_path input --dest_path ./output

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
! rm -rf .zen
! zenml init

⠋ Initializing ZenML repository at /kaggle/working.
⠙ Initializing ZenML repository at /kaggle/working.
⠹ Initializing ZenML repository at /kaggle/working.
⠸ Initializing ZenML repository at /kaggle/working.
⠼ Initializing ZenML repository at /kaggle/working.
⠴ Initializing ZenML repository at /kaggle/working.
⠦ Initializing ZenML repository at /kaggle/working.
⠧ Initializing ZenML repository at /kaggle/working.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
⠇ Initializing ZenML repository at /kaggle/working.
⠏ Initializing ZenML repository at /kaggle/working.
⠋ Initializing ZenML repository at /kaggle/working.
⠙ Initializing ZenML repository at /kaggle/working.
⠹ Initializing ZenML repository at /kaggle/working.
⠸ Initializing ZenML repository at /kaggle/working.
⠼ Initializing ZenML repository at /kaggle/working.
⠴ Initializing ZenML repository at /kaggle/working.
ZenML repository initialized at /kaggle/working.
⠦ Initializing ZenML reposito

In [7]:
import os
import pickle
import scipy
import numpy

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from zenml.steps import step, Output
from zenml.pipelines import pipeline

def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

@step
def importer() -> Output(
    X_train=scipy.sparse.csr.csr_matrix,
    X_test=scipy.sparse.csr.csr_matrix,
    y_train=numpy.ndarray,
    y_test=numpy.ndarray,
):
    X_train, y_train = load_pickle(os.path.join("./output", "train.pkl"))
    X_val, y_val = load_pickle(os.path.join("./output", "val.pkl"))
    return X_train, X_val, y_train, y_val

@step
def trainer(X_train: scipy.sparse.csr.csr_matrix, y_train: numpy.ndarray) -> RandomForestRegressor:
    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    return rf

@step
def evaluator(X_test: scipy.sparse.csr.csr_matrix, y_test: numpy.ndarray, model: RandomForestRegressor) -> numpy.float64:
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f"RMSE: {rmse}")   
    return rmse

@pipeline(enable_cache=False)
def pipeline(importer, trainer, evaluator):
    X_train, X_test, y_train, y_test = importer()
    model = trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

pipeline(
    importer=importer(), trainer=trainer(), evaluator=evaluator()
).run(unlisted=True)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Running unlisted pipeline on stack default (caching disabled)
Step importer has started.
Step importer has finished in 3.505s.
Step trainer has started.
Step trainer has finished in 12.521s.
Step evaluator has started.
RMSE: 2.453983836538874
Step evaluator has finished in 0.915s.
Pipeline run pipeline-2023_05_24-19_06_26_738161 has finished in 19.105s.
Dashboard URL: https://bsenst-zenml-server.hf.space/pipelines/all-runs


In [8]:
# Register the MLflow experiment tracker
# !zenml experiment-tracker register mlflow_tracker --flavor=mlflow

# Create a new stack that includes an MLflow experiment
# !zenml stack register mlflow_exp_tracker_stack -a default -o default -e mlflow_tracker

# Set the new stack as active
!zenml stack set mlflow_exp_tracker_stack

Connected to the ZenML server: 'https://bsenst-zenml-server.hf.space'
Running with active workspace: 'default' (repository)
Active repository stack set to: 'mlflow_exp_tracker_stack'.
⠧ Setting the repository active stack to 'mlflow_exp_tracker_stack'...racker_stack'...


In [9]:
# https://github.com/zenml-io/zenbytes
from absl import logging as absl_logging
import warnings

warnings.filterwarnings("ignore")
absl_logging.set_verbosity(-10000)

In [10]:
! zenml status

Using configuration from: '/root/.config/zenml'
Local store files are located at: '/root/.config/zenml/local_stores'
Active repository root: /kaggle/working
Connected to a ZenML server: 'https://bsenst-zenml-server.hf.space'
The current user is: 'Benjamin'
The active workspace is: 'default' (repository)
The active stack is: 'mlflow_exp_tracker_stack' (repository)


In [11]:
! zenml integration install mlflow wandb -y

import mlflow

@step(enable_cache=False, experiment_tracker="mlflow_tracker")
def trainer(X_train: scipy.sparse.csr.csr_matrix, y_train: numpy.ndarray) -> RandomForestRegressor:
    mlflow.sklearn.autolog()  # log all model hparams and metrics to MLflow
    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    return rf

pipeline(
    importer=importer(), trainer=trainer(), evaluator=evaluator()
).run(unlisted=True)

⠧ Installing integrations.....
Reloading configuration file /kaggle/working/.zen/config.yaml
Running unlisted pipeline on stack mlflow_exp_tracker_stack (caching disabled)
Step importer has started.
Step importer has finished in 3.089s.
Step trainer has started.


2023/05/24 19:07:38 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


Step trainer has finished in 25.649s.
Step evaluator has started.
RMSE: 2.453983836538874
Step evaluator has finished in 0.901s.
Pipeline run pipeline-2023_05_24-19_07_32_893158 has finished in 32.066s.
Dashboard URL: https://bsenst-zenml-server.hf.space/pipelines/all-runs


In [12]:
! zenml stack list

Connected to the ZenML server: 'https://bsenst-zenml-server.hf.space'
Running with active workspace: 'default' (repository)
⠋ Listing stacks...
⠙ Listing stacks...
┏━━━━━━━━┯━━━━━━━━┯━━━━━━━━┯━━━━━━━━┯━━━━━━━━┯━━━━━━━━┯━━━━━━━┯━━━━━━━━┯━━━━━━━┓
┃        │        │        │        │        │ ARTIFA │ ORCHE │ EXPERI │ MODEL ┃
┃        │ STACK  │ STACK  │        │        │ CT_STO │ STRAT │ MENT_T │ _DEPL ┃
┃ ACTIVE │ NAME   │ ID     │ SHARED │ OWNER  │ RE     │ OR    │ RACKER │ OYER  ┃
┠────────┼────────┼────────┼────────┼────────┼────────┼───────┼────────┼───────┨
┃        │ defaul │ af0af3 │ ➖     │ Benjam │ defaul │ defau │        │       ┃
┃        │ t      │ 67-422 │        │ in     │ t      │ lt    │        │       ┃
┃        │        │ 6-47e8 │        │        │        │       │        │       ┃
┃        │        │ -9345- │        │        │        │       │        │       ┃
┃        │        │ a63e45 │        │        │        │       │        │       ┃
┃        │        │ a0693d 

In [13]:
# ! zenml stack set mlflow_exp_tracker_stack

In [14]:
from zenml.post_execution import get_unlisted_runs

pipeline_run = get_unlisted_runs()[0]
step = pipeline_run.get_step("trainer")
experiment_tracker_url = step.metadata["experiment_tracker_url"].value

print(experiment_tracker_url)

file:/root/.config/zenml/local_stores/498be34b-aee3-4ef0-afc1-02c8c05ad241/mlruns


In [15]:
from zenml.integrations.mlflow.steps import (
    MLFlowDeployerParameters,
    mlflow_model_deployer_step
)
from zenml.pipelines import pipeline
from zenml.steps import step


@step
def deployment_trigger(test_acc: float) -> bool:
    """Only deploy if the test accuracy > 90%."""
    return test_acc > 0.9

@pipeline(enable_cache=False)
def train_evaluate_deploy_pipeline(
    importer,
    trainer,
    evaluator,
    deployment_trigger,
    model_deployer,
):
    """Train and deploy a model with MLflow."""
    X_train, X_test, y_train, y_test = importer()
    model = trainer(X_train=X_train, y_train=y_train)
    test_acc = evaluator(X_test=X_test, y_test=y_test, model=model)
    deployment_decision = deployment_trigger(test_acc)  # new
    model_deployer(deployment_decision, model)  # new


mlflow_pipeline = train_evaluate_deploy_pipeline(
    importer=importer(),
    trainer=trainer(),
    evaluator=evaluator(),
    deployment_trigger=deployment_trigger(),  # new
    model_deployer=mlflow_model_deployer_step(
        MLFlowDeployerParameters()
    ),  # new
)

In [16]:
# # Define MLflow experiment tracker from last lesson
# !zenml experiment-tracker register mlflow_tracker --flavor=mlflow

# # Register the MLflow model deployer
# ! zenml model-deployer register mlflow --flavor=mlflow

# # Create a new stack with MLflow components
# !zenml stack register mlflow_stack -a default -o default -d mlflow -e mlflow_tracker

# Set the new stack as active
!zenml stack set mlflow_stack

Connected to the ZenML server: 'https://bsenst-zenml-server.hf.space'
Running with active workspace: 'default' (repository)
Active repository stack set to: 'mlflow_stack'.
⠧ Setting the repository active stack to 'mlflow_stack'...mlflow_stack'...


In [17]:
mlflow_pipeline.run(unlisted=True, enable_cache=False)

⠴ Starting service 'MLFlowDeploymentService[28e6f6c5-5336-4721-8d35-9d9e3b0c5494] (type: model-serving, flavor: 
mlflow)'.

⠴ Starting service 'MLFlowDeploymentService[28e6f6c5-5336-4721-8d35-9d9e3b0c5494] (type: model-serving, flavor: 
mlflow)'.

MLflow deployment service started and reachable at:
    http://127.0.0.1:8000/invocations

Step model_deployer has finished in 30.272s.
Pipeline run train_evaluate_deploy_pipeline-2023_05_24-19_08_37_804916 has finished in 59.359s.
Dashboard URL: https://bsenst-zenml-server.hf.space/pipelines/all-runs


In [18]:
!zenml model-deployer models list

Connected to the ZenML server: 'https://bsenst-zenml-server.hf.space'
Running with active workspace: 'default' (repository)
Running with active stack: 'mlflow_stack' (repository)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
┏━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━┓
┃        │                  │                  │ PIPELINE_STEP_NA │            ┃
┃ STATUS │ UUID             │ PIPELINE_NAME    │ ME               │ MODEL_NAME ┃
┠────────┼──────────────────┼──────────────────┼──────────────────┼────────────┨
┃   ✅   │ 28e6f6c5-5336-47 │ train_evaluate_d │ model_deployer   │ model      ┃
┃        │ 21-8d35-9d9e3b0c │ eploy_pipeline   │                  │            ┃
┃        │ 5494             │                  │                  │            ┃
┗━━━━

In [19]:
from zenml.post_execution import get_unlisted_runs

last_run = get_unlisted_runs()[0]
deployer_step = last_run.get_step("model_deployer")
deployed_model_url = deployer_step.metadata["deployed_model_url"].value
print(deployed_model_url)

http://127.0.0.1:8000/invocations
